In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
next_layer = "silver"

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "")
environment = dbutils.widgets.get("environment")
print(proc_date)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
# Dynamic select query to read the data for specific proc_date
dynamic_query = f""" CREATE OR REPLACE TEMP VIEW temp_pos_wcm_saleout AS SELECT *
FROM udp_wcm_pro.udp_wcm_bronze_pro.pos_wcm_saleout
WHERE ProcDate like '{str(proc_date).replace('-', '')}%'
"""

# Execute the query
spark.sql(dynamic_query)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_promotion
(
hash_id BIGINT,
  receipt_no STRING,
  pos_no STRING,
  store_id STRING,
  calday DATE,
  caltime STRING,
  tran_no STRING,
  uom STRING,
  line_no STRING,
  offer_no STRING,
  offer_type STRING,
  item_no STRING,
  item_qty Double,
  discount_amount Double,
  proc_date TIMESTAMP
) 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'true',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'true'
)
""")

In [0]:
spark.sql(
    f"""
CREATE OR REPLACE TEMP VIEW view_pos_wcm_saleout_promotion as
WITH dat AS (
    SELECT
  LTRIM('0',main_tbl.ReceiptNo) receipt_no,
  LTRIM('0',main_tbl.PosNo) pos_no,
  LTRIM('0',main_tbl.StoreCode) store_id,
  to_date(to_timestamp(main_tbl.CalendarDay, 'MM/dd/yyyy HH:mm:ss')) AS calday,
  date_format(to_timestamp(substr(main_tbl.TranTime, 1, 6), 'HHmmss'),'HH:mm:ss') AS caltime,
  trans.TranNo tran_no,
  trans.UOM,
  discount.LineNo line_no,
  discount.OfferNo offer_no,
  discount.OfferType offer_type,
  discount.ItemNo item_no,
  discount.Quantity item_qty,
  discount.DiscountAmount discount_amount,
  TO_TIMESTAMP(main_tbl.ProcDate, 'yyyyMMddHHmmss') AS proc_date
  FROM temp_pos_wcm_saleout main_tbl 
  LATERAL VIEW EXPLODE(main_tbl.TransLine) AS trans
  LATERAL VIEW EXPLODE(trans.DiscountEntry) AS discount
  WHERE main_tbl.ReceiptNo = discount.ReceiptNo
    AND trans.TranNo = discount.TranNo
)
SELECT 
    farm_fingerprint(CONCAT(
        IFNULL(CAST(receipt_no AS STRING), ""),
        IFNULL(CAST(tran_no AS STRING), ""),
        IFNULL(CAST(line_no AS STRING), ""),
        IFNULL(CAST(store_id AS STRING), "")
    )) AS hash_id,
    *
FROM dat
QUALIFY ROW_NUMBER() OVER (PARTITION BY receipt_no, tran_no, line_no, store_id ORDER BY proc_date DESC) = 1    
"""
)

In [0]:
spark.sql(f"DELETE FROM {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_promotion WHERE RECEIPT_NO IN (SELECT RECEIPT_NO FROM view_pos_wcm_saleout_promotion)")

In [0]:
spark.sql(f"INSERT INTO {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_promotion SELECT * FROM view_pos_wcm_saleout_promotion")